In [1]:
from pathlib import Path
import pandas as pd
import torch
import torch.nn as nn

data_path = "/home/takeru/AlphaSymbol/data/prfndim/d3-a2-c3-r3-status.csv"
data_path = Path(data_path)
df = pd.read_csv(data_path)
batch_size = len(df)

In [10]:
max_value = 20 # max value of input and output
pad_idx = max_value + 1
sos_idx = max_value + 2
eos_idx = max_value + 3
token_num = max_value + 4

sequence = []
# organize input and output data
for input, output in zip(df["input"], df["output"]): 
    input, output = eval(input), eval(output)
    point_li = []
    for x, y in zip(input, output):
        point_li.append([sos_idx, *x, y, eos_idx])
    sequence.append(point_li)

# pad
max_input_size = max([len(seq[0]) for seq in sequence])
for i, seq in enumerate(sequence):
    current_len = len(seq[0])
    for p in seq:
        p.extend([pad_idx] * (max_input_size- current_len))

point_num = len(sequence[0])
sequence = torch.tensor(sequence) # shape is (batch, point_num, input_size)
sequence = sequence.permute(1, 0, 2) # shape is (seq, batch, token_id)

In [11]:
print(sequence.shape)

torch.Size([10, 28, 5])


In [15]:
token_emb_dim = 64
m = nn.Embedding(num_embeddings=token_num, embedding_dim=token_emb_dim, padding_idx=pad_idx)
nn.init.normal_(m.weight, mean=0, std=token_emb_dim**-0.5)
if pad_idx is not None:
    nn.init.constant_(m.weight[pad_idx], 0)
print(sequence.shape)
sequences_embeddings = m(sequence)
print(sequences_embeddings.shape)
sequences_embeddings = sequences_embeddings.view(point_num, batch_size, -1)
print(sequences_embeddings.shape)

output_dim = 512
hidden_size = max_len * token_emb_dim
fc = nn.Linear(hidden_size, output_dim)
output = fc(sequences_embeddings)
print(output.shape)

torch.Size([10, 28, 5])
torch.Size([10, 28, 5, 64])
torch.Size([10, 28, 320])
torch.Size([10, 28, 512])
